In [10]:
import langchain

In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import Pinecone
import uuid

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents
extracted_data = load_pdf("data/")

KeyboardInterrupt: 

In [28]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

NameError: name 'extracted_data' is not defined

In [6]:
PINECONE_API_KEY = "9f3cbd34-0ab3-41b6-bf1f-ead705825563"
PINECONE_API_ENV = "us-east-1"

In [7]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Download the embeddings
embeddings = download_hugging_face_embeddings()

In [8]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [9]:
pc = Pinecone(
    api_key=PINECONE_API_KEY  
)

In [10]:
# Initialize Pinecone
from pinecone import ServerlessSpec
spec = ServerlessSpec(cloud = "aws", region=PINECONE_API_ENV)

In [11]:
import time

index_name = "medical-chatbot"

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of embed 3
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 7020}},
 'total_vector_count': 7020}

In [37]:
def store_embeddings_in_pinecone(text_chunks, embeddings):
    for chunk in text_chunks:
        # Generate embeddings for each chunk
        chunk_text = chunk.page_content
        embedding = embeddings.embed_documents([chunk_text])[0]
        
        # Generate a unique ID for the chunk
        chunk_id = str(uuid.uuid4())
        
        # Upsert the embedding and chunk into Pinecone
        index.upsert([(chunk_id, embedding, {"text": chunk_text})])
# Store the embeddings and chunks in Pinecone
store_embeddings_in_pinecone(text_chunks, embeddings)
print("Embeddings and chunks stored in Pinecone.")

NameError: name 'text_chunks' is not defined

In [12]:
query = "What are Allergies"
query_embedding = embeddings.embed_documents([query])[0]

# Perform the similarity search
result = index.query( vector=query_embedding,
        top_k=3,
        include_metadata=True)

In [13]:
#result

In [14]:
chunks = []
for m in result["matches"]:
    content = m["metadata"]["text"]
    chunks.append(content)

In [15]:
chunks[0]

"GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"

In [16]:
query = "What are Allergies"
query_embedding = embeddings.embed_documents([query])[0]

# Perform the similarity search
result = index.query( vector=query_embedding,
        top_k=3,
        include_metadata=True)

In [17]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [18]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [19]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [20]:
def retrieve_documents(query, top_k=2):
    query_vector = embeddings.embed_documents([query])[0]
    result = index.query(queries=[query_vector], top_k=top_k)
    #return [match['metadata']['text'] for match in result['matches']]
    return result

In [21]:
from langchain.vectorstores import Pinecone as PineconeStore
vectordb = PineconeStore(index, embeddings.embed_query,"text")

In [22]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_kwargs={'k': 2}),
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True
)

In [23]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

ValueError: The argument order for `query()` has changed; please use keyword arguments instead of positional arguments. Example: index.query(vector=[0.1, 0.2, 0.3], top_k=10, namespace='my_namespace')

In [50]:
answer = input("Type 'y' to confirm deletion of the index...\n>> ")
if answer == "y":
    pc.delete_index(index_name)
    print("Index Deleted!")
else:
    print("Deletion Cancelled")

Deletion Cancelled
